In [1]:
import os 
import sys
import pandas as pd
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)
from h2_station_distributor.models.stations import Stations
from h2_station_distributor.utils.helpers  import Data
stations = Stations()
reqs_df = pd.read_csv(Data.find_file("reqs_df.csv"))

In [3]:
stations.solution()

In [2]:
def run_algo(reqs_df):
  fin_df = pd.DataFrame({"point":[], "coords":[], "mindist":[], "trucks":[], "fitness":[], "size":[], "region":[]})
  for i in range(reqs_df.shape[0]):
    region = reqs_df.iloc[i,0]
    small = reqs_df.iloc[i,1]
    medium = reqs_df.iloc[i,2]
    big = reqs_df.iloc[i,3]
    print(region, small, medium, big)
    solution = stations.solution(region,big, medium, small)
    fin_df = pd.concat([fin_df, solution]).reset_index(drop=True)
  return fin_df

run_algo(reqs_df)

Auvergne-Rhône-Alpes 48 32 15


/Users/youssefjouini/opt/anaconda3/envs/sust/lib/python3.9/site-packages/geopandas/geodataframe.py:1443: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super().__setitem__(key, value)
/Users/youssefjouini/opt/anaconda3/envs/sust/lib/python3.9/site-packages/geopandas/geodataframe.py:1443: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super().__setitem__(key, value)


IndexError: index 382 is out of bounds for axis 0 with size 382